In [1]:
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import warnings

from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, auc, mean_squared_error, roc_curve, confusion_matrix, precision_score, recall_score, f1_score,\
log_loss, roc_auc_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
import warnings
from scipy.stats import chi2_contingency


from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import  DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split , StratifiedKFold  , StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
import os
import random
from sklearn.metrics import f1_score, fbeta_score, make_scorer, balanced_accuracy_score, accuracy_score, roc_auc_score
pd.set_option("display.max_columns",30)

# random.seed(40)
# np.random.rand(40)

rand = 40

In [4]:
data = pd.read_csv('all_data_processed_finaleng_2.csv')
ss = pd.read_csv('SampleSubmission.csv')

data = data.drop(['Unnamed: 0'], axis = 1)

In [5]:
data.head()

,ID,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,target,tag,Policy Start Date year,Policy Start Date month,Policy Start Date day,Policy Start Date dayofweek,...,Orange,Wine,White,Blue,Cream,Car_Category_new,age_group,age_group_risk,Policy_brackets,State_zone,is_rainfall,is_peak,risk_index,Gender_age_risk,No Pol/Age
0,ID_0040R73,Male,30.0,1,Saloon,TOYOTA,NaN,NaN,Car Classic,0.0,train,2010.0,5,14,4,...,0,0,0,0,0,Car,25-34,low-risk mature,12months,NaN,0,1,2,Male- low-risk mature,0.033333
1,ID_0046BNK,Female,79.0,1,JEEP,TOYOTA,NaN,NaN,Car Classic,1.0,train,2010.0,11,29,0,...,0,0,0,0,0,JEEP,65+,High-risk elderly,12months,NaN,0,0,0,Female- High-risk elderly,0.012658
2,ID_005QMC3,Male,43.0,1,Saloon,TOYOTA,NaN,NaN,Car Classic,0.0,train,2010.0,3,21,6,...,0,0,0,0,0,Car,35-44,low-risk mature,12months,NaN,0,1,2,Male- low-risk mature,0.023256
3,ID_0079OHW,Male,NaN,1,NaN,NaN,NaN,NaN,CarSafe,0.0,train,2010.0,8,21,5,...,0,0,0,0,0,NaN,NaN,NaN,12months,NaN,0,0,0,NaN,NaN
4,ID_00BRP63,NaN,20.0,3,NaN,NaN,ikeja,Lagos,Muuve,1.0,train,2010.0,8,29,6,...,0,0,0,0,0,NaN,18-20,high-risk youngsters,6months,South_West,1,0,1,NaN,0.150000


In [6]:
data.columns

Index(['ID', 'Gender', 'Age', 'No_Pol', 'Car_Category', 'Subject_Car_Make',
       'LGA_Name', 'State', 'ProductName', 'target', 'tag',
       'Policy Start Date year', 'Policy Start Date month',
       'Policy Start Date day', 'Policy Start Date dayofweek',
       'Policy End Date year', 'Policy End Date month', 'Policy End Date day',
       'Policy End Date dayofweek', 'weekinmonth', 'Policy_delta',
       'Next 1 Months', 'Next 1 Quarters', 'Next 2 Months', 'Next 2 Quarters',
       'Next 3 Months', 'Next 3 Quarters', 'summer', 'winter', 'spring',
       'autumn', 'Policy_isfestive', 'color_group', 'Yellow', 'Bright',
       'Black', 'Gold', 'Dark', 'Purple', 'Silver', 'Ash', 'Red', 'Light',
       'Grey', 'Brown', 'Green', 'Orange', 'Wine', 'White', 'Blue', 'Cream',
       'Car_Category_new', 'age_group', 'age_group_risk', 'Policy_brackets',
       'State_zone', 'is_rainfall', 'is_peak', 'risk_index', 'Gender_age_risk',
       'No Pol/Age'],
      dtype='object')

In [7]:
data.dtypes

ID                  object
Gender              object
Age                float64
No_Pol               int64
Car_Category        object
                    ...   
is_rainfall          int64
is_peak              int64
risk_index           int64
Gender_age_risk     object
No Pol/Age         float64
Length: 61, dtype: object

In [8]:
data['State_Statezone'] = data['State'] + "_" + data['State_zone']

,ID,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,target,tag,Policy Start Date year,Policy Start Date month,Policy Start Date day,Policy Start Date dayofweek,...,Wine,White,Blue,Cream,Car_Category_new,age_group,age_group_risk,Policy_brackets,State_zone,is_rainfall,is_peak,risk_index,Gender_age_risk,No Pol/Age,State_Statezone
0,ID_0040R73,Male,30.0,1,Saloon,TOYOTA,NaN,NaN,Car Classic,0.0,train,2010.0,5,14,4,...,0,0,0,0,Car,25-34,low-risk mature,12months,NaN,0,1,2,Male- low-risk mature,0.033333,NaN
1,ID_0046BNK,Female,79.0,1,JEEP,TOYOTA,NaN,NaN,Car Classic,1.0,train,2010.0,11,29,0,...,0,0,0,0,JEEP,65+,High-risk elderly,12months,NaN,0,0,0,Female- High-risk elderly,0.012658,NaN
2,ID_005QMC3,Male,43.0,1,Saloon,TOYOTA,NaN,NaN,Car Classic,0.0,train,2010.0,3,21,6,...,0,0,0,0,Car,35-44,low-risk mature,12months,NaN,0,1,2,Male- low-risk mature,0.023256,NaN
3,ID_0079OHW,Male,NaN,1,NaN,NaN,NaN,NaN,CarSafe,0.0,train,2010.0,8,21,5,...,0,0,0,0,NaN,NaN,NaN,12months,NaN,0,0,0,NaN,NaN,NaN
4,ID_00BRP63,NaN,20.0,3,NaN,NaN,ikeja,Lagos,Muuve,1.0,train,2010.0,8,29,6,...,0,0,0,0,NaN,18-20,high-risk youngsters,6months,South_West,1,0,1,NaN,0.150000,Lagos_South_West


In [11]:
drop_cols = ['Next 2 Quarters', 'Next 3 Quarters', 'Next 1 Quarters', \
            'Policy Start Date year', 'Car_Category_new',\
        'Policy End Date month', 'Policy Start Date day', 'Policy Start Date month',\
            'Next 1 Months', 'Next 2 Months', 'Next 3 Months', 'Next 2 Quarters', 
           'age_group_risk', 'Car_Category']
             

In [12]:
data = data.drop(drop_cols, axis = 1)

In [13]:
data = data.drop(['Yellow', 'Orange', 'Light', 'Dark', 'Bright', 'Purple', 'Cream'], axis = 1)

In [14]:
#Extract categorical

cat_cols = []

for col in data.columns:
    
    if data[col].dtype == 'O':
        
        cat_cols.append(col)

In [15]:
cat_cols.remove('tag')
cat_cols.remove('ID')

In [16]:
data.isnull().sum()

ID                                0
Gender                         1246
Age                             589
No_Pol                            0
Subject_Car_Make               2785
LGA_Name                       7145
State                          7145
ProductName                       0
target                         1202
tag                               0
Policy Start Date dayofweek       0
Policy End Date year              6
Policy End Date day               0
Policy End Date dayofweek         0
weekinmonth                       0
Policy_delta                     11
summer                            0
winter                            0
spring                            0
autumn                            0
Policy_isfestive                  0
color_group                    8214
Black                             0
Gold                              0
Silver                            0
Ash                               0
Red                               0
Grey                        

In [17]:
for col in cat_cols:

    data[col] = pd.factorize(data[col])[0]

In [18]:
for col in cat_cols:
    
    print(data[col].value_counts())

 0    8356
 1    3679
-1    1246
Name: Gender, dtype: int64
 0     5466
-1     2785
 6     1146
 2      654
 7      577
 3      512
 14     375
 9      307
 11     275
 10     133
 4      129
 13     121
 15     102
 8       69
 17      68
 24      66
 26      47
 23      47
 20      43
 1       42
 21      35
 29      33
 18      30
 12      29
 25      26
 5       21
 33      18
 28      17
 22      16
 16      12
 30       9
 32       8
 40       8
 35       7
 19       6
 36       6
 37       6
 38       6
 31       5
 27       5
 39       5
 41       5
 34       4
Name: Subject_Car_Make, dtype: int64
-1      7145
 3      1284
 0       625
 11      306
 8       257
        ... 
 76        4
 103       4
 99        4
 98        4
 112       4
Name: LGA_Name, Length: 131, dtype: int64
-1     7145
 0     4073
 1      693
 4      340
 6      204
 2      138
 3      120
 5      117
 10      78
 8       56
 13      50
 7       49
 12      47
 17      43
 14      24
 11      24
 15      1

In [19]:

data.head()

,ID,Gender,Age,No_Pol,Subject_Car_Make,LGA_Name,State,ProductName,target,tag,Policy Start Date dayofweek,Policy End Date year,Policy End Date day,Policy End Date dayofweek,weekinmonth,...,Grey,Brown,Green,Wine,White,Blue,age_group,Policy_brackets,State_zone,is_rainfall,is_peak,risk_index,Gender_age_risk,No Pol/Age,State_Statezone
0,ID_0040R73,0,30.0,1,0,-1,-1,0,0.0,train,4,2011.0,13,4,3,...,0,0,0,0,0,0,0,0,-1,0,1,2,0,0.033333,-1
1,ID_0046BNK,1,79.0,1,0,-1,-1,0,1.0,train,0,2011.0,28,0,4,...,1,0,0,0,0,0,1,0,-1,0,0,0,1,0.012658,-1
2,ID_005QMC3,0,43.0,1,0,-1,-1,0,0.0,train,6,2011.0,20,6,4,...,0,0,0,0,0,0,2,0,-1,0,1,2,0,0.023256,-1
3,ID_0079OHW,0,NaN,1,-1,-1,-1,1,0.0,train,5,2011.0,20,5,4,...,0,0,0,0,0,0,-1,0,-1,0,0,0,-1,NaN,-1
4,ID_00BRP63,-1,20.0,3,-1,0,0,2,1.0,train,6,2010.0,31,4,4,...,0,0,0,0,0,0,3,1,0,1,0,1,-1,0.150000,0


In [20]:
train = data[data['tag'] == 'train']

test = data[data['tag'] == 'test']


train = train.drop(['tag', 'ID'], axis = 1)

test = test.drop(['tag'], axis = 1)

In [21]:
train.shape, test.shape

((12079, 40), (1202, 41))

In [22]:
y = train['target'].astype(int)

X = train.drop(['target'], axis = 1)

# Modelling

In [23]:
X.columns

Index(['Gender', 'Age', 'No_Pol', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'Policy Start Date dayofweek', 'Policy End Date year',
       'Policy End Date day', 'Policy End Date dayofweek', 'weekinmonth',
       'Policy_delta', 'summer', 'winter', 'spring', 'autumn',
       'Policy_isfestive', 'color_group', 'Black', 'Gold', 'Silver', 'Ash',
       'Red', 'Grey', 'Brown', 'Green', 'Wine', 'White', 'Blue', 'age_group',
       'Policy_brackets', 'State_zone', 'is_rainfall', 'is_peak', 'risk_index',
       'Gender_age_risk', 'No Pol/Age', 'State_Statezone'],
      dtype='object')

In [24]:
cat_cols = cat_cols + ['risk_index']

In [25]:
#Cross-Validation LGBM routine

def lgb_crossval(model):

    rand = 40

    fold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= rand)

    i = 1

    f1_bin = []
    
    f1_mac = []

    acc = []

    bal_acc = []
    
    mcc = []
    
    test_new = test[X.columns]
    
    predictions = []
    
    
    
    # Initialize an empty array to hold feature importances
    feature_importances = np.zeros(X.shape[1])

    




    for train_index, test_index in fold.split(X, y):
        
    
        print('fold n°: ', i)

        x_data, x_val = X.iloc[train_index], X.iloc[test_index]

        y_data, y_val = y.iloc[train_index], y.iloc[test_index]
    
    
    
   
        
        model.fit(x_data, y_data, categorical_feature = cat_cols, eval_set= [(x_val, y_val)],  verbose = 0)
    
        y_train_pred = model.predict(x_data)
        
        y_test_pred = model.predict(x_val)
        
        predictions.append(model.predict(test_new))   
        
        
        feature_importances += model.feature_importances_ 
    
        print('f1_binary train', f1_score(y_data, y_train_pred, average = 'binary'))
        print('f1_binary test',  f1_score(y_val, y_test_pred, average = 'binary'))
        
        print('f1_macro train', f1_score(y_data, y_train_pred, average = 'macro'))
        print('f1_macro test',  f1_score(y_val, y_test_pred, average = 'macro'))
    
        print('accuracy train', accuracy_score(y_data, y_train_pred))
        print('accuracy test',  accuracy_score(y_val, y_test_pred))
    
        print('balanced accuracy train', balanced_accuracy_score(y_data, y_train_pred))
        print('balanced accuracy test',  balanced_accuracy_score(y_val, y_test_pred))
    
    
        print('mcc train', matthews_corrcoef(y_data, y_train_pred))
        print('mcc test', matthews_corrcoef(y_val, y_test_pred))
    
    
    
        f1_bin.append(f1_score(y_val, y_test_pred, average = 'binary'))
        
        f1_mac.append(f1_score(y_val, y_test_pred, average = 'macro'))
        
        acc.append(accuracy_score(y_val, y_test_pred))
    
        bal_acc.append(balanced_accuracy_score(y_val, y_test_pred))
        
        mcc.append(matthews_corrcoef(y_val, y_test_pred))


        i = i + 1
        
    print('Mean f1 binary test: ', np.mean(f1_bin)) 
    
    print('Mean f1 macro test: ', np.mean(f1_mac))

    print('Mean Acc test: ', np.mean(acc))

    print('Mean Bal Acc test: ', np.mean(bal_acc))
    
    print('Mean Mcc test: ', np.mean(mcc))
    
    
    print('STD f1 binary test: ', np.std(f1_bin)) 
    
    print('STD Mean acc test: ', np.std(acc))
    
    return predictions, feature_importances/10, f1_bin

In [26]:
lgb_model = LGBMClassifier(n_estimators = 1000, learning_rate = 0.09, scale_pos_weight = 7.3, \
                               boosting = 'dart', num_leaves = 60, \
                               importance_type = 'gain', reg_lambda = 0.5,
                          random_state = 40)

In [27]:
predictions, importances, f1_scores = lgb_crossval(lgb_model)

fold n°:  1
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

f1_binary train 0.7941800545619885
f1_binary test 0.40588235294117647
f1_macro train 0.8786819591792754
f1_macro test 0.6542899240621105
accuracy train 0.9375402446877013
accuracy test 0.8327814569536424
balanced accuracy train 0.9644911620123418
balanced accuracy test 0.6786648068251857
mcc train 0.7822075104972456
mcc test 0.3156440555802979
fold n°:  2
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

f1_binary train 0.7961106046794287
f1_binary test 0.41566265060240964
f1_macro train 0.8798720060414107
f1_macro test 0.661286219734986
accuracy train 0.9382761475485236
accuracy test 0.8394039735099338
balanced accuracy train 0.9649095282920197
balanced accuracy test 0.6824277419145555
mcc train 0.7841382361611633
mcc test 0.32782180500657504
fold n°:  3
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


...

f1_binary train 0.7957446808510639
f1_binary test 0.3687150837988827
f1_macro train 0.8796629322312983
f1_macro test 0.6294498645427844
accuracy train 0.9381841596909208
accuracy test 0.8129139072847682
balanced accuracy train 0.9648609077598829
balanced accuracy test 0.657289167505095
mcc train 0.7837979013651662
mcc test 0.269568968035245
fold n°:  9
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

f1_binary train 0.7923728813559322
f1_binary test 0.4057142857142857
f1_macro train 0.8775835553324839
f1_macro test 0.6525183238832803
accuracy train 0.9368963296844817
accuracy test 0.8278145695364238
balanced accuracy train 0.9641288433382138
balanced accuracy test 0.680532982483296
mcc train 0.7804276547126727
mcc test 0.3141545099501675
fold n°:  10
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

f1_binary train 0.7898703647874585
f1_binary test 0.35294117647058826
f1_macro train 0.8760227169897026
f1_macro test 0.6234329797492768
accuracy train 0.9358903605592347
accuracy test 0.8177299088649544
balanced accuracy train 0.9635536498640451
balanced accuracy test 0.6433372296902397
mcc train 0.7779057145754921
mcc test 0.2532373147396016
Mean f1 binary test:  0.3837147128981404
Mean f1 macro test:  0.640911707685078
Mean Acc test:  0.825150474330204
Mean Bal Acc test:  0.6639287147263195
Mean Mcc test:  0.28907277183034663
STD f1 binary test:  0.019824935865866745
STD Mean acc test:  0.008485958823177528


# LGBM 2



In [28]:
#LGB JULY DART ALPHA

lgb_model_2 = LGBMClassifier(n_estimators = 1000, learning_rate = 0.09, scale_pos_weight = 7.3, boosting = 'dart', \
                            num_leaves = 60, importance_type = 'gain', reg_alpha = 0.5,
                            random_state = 40, feature_fraction = 0.9)

In [29]:
predictions, importances, f1_scores = lgb_crossval(lgb_model_2)

fold n°:  1
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

f1_binary train 0.791779993955878
f1_binary test 0.391812865497076
f1_macro train 0.8772006897571936
f1_macro test 0.6457617847253943
accuracy train 0.9366203661116732
accuracy test 0.8278145695364238
balanced accuracy train 0.9639682041627444
balanced accuracy test 0.6698867875563629
mcc train 0.7798092065707621
mcc test 0.2989038729993178
fold n°:  2
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

f1_binary train 0.7978075517661388
f1_binary test 0.41194029850746267
f1_macro train 0.8809169950834161
f1_macro test 0.6586371362792
accuracy train 0.9389200625517432
accuracy test 0.8369205298013245
balanced accuracy train 0.9652755987867379
balanced accuracy test 0.6810166412560418
mcc train 0.7858365236841918
mcc test 0.32318295078980913
fold n°:  3
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


...

f1_binary train 0.7906976744186047
f1_binary test 0.37183098591549296
f1_macro train 0.8765489891272666
f1_macro test 0.6318155414778823
accuracy train 0.9362524146812621
accuracy test 0.8153973509933775
balanced accuracy train 0.9637628111273793
balanced accuracy test 0.6587015968836262
mcc train 0.778754814241963
mcc test 0.2735231513618058
fold n°:  9
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

f1_binary train 0.7930930021205695
f1_binary test 0.3977272727272727
f1_macro train 0.8780279825417663
f1_macro test 0.6475070472163496
accuracy train 0.93717229325729
accuracy test 0.8245033112582781
balanced accuracy train 0.9642857142857143
balanced accuracy test 0.6756958955705182
mcc train 0.7811470815744042
mcc test 0.3045529615483107
fold n°:  10
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

f1_binary train 0.7903469079939669
f1_binary test 0.33727810650887574
f1_macro train 0.8763173033648276
f1_macro test 0.6146891495935516
accuracy train 0.9360743193524651
accuracy test 0.8144159072079536
balanced accuracy train 0.9636582304957122
balanced accuracy test 0.6325215923111891
mcc train 0.7783813897951534
mcc test 0.235106842352615
Mean f1 binary test:  0.3848311969675132
Mean f1 macro test:  0.6410860028734419
Mean Acc test:  0.8240740410519212
Mean Bal Acc test:  0.6656994320830133
Mean Mcc test:  0.2902848989648791
STD f1 binary test:  0.02094805903159251
STD Mean acc test:  0.007389372729361304


# ENSEMBLE

In [30]:
#Create LIGHTGBM ENSEMBLE

def ensemb_crossval():

    rand = 40

    fold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= rand)

    i = 1

    f1_bin = []
    
    f1_mac = []

    acc = []

    bal_acc = []
    
    mcc = []
    
    test_new = test[X.columns]
    
    predictions = []
    
    
    

    




    for train_index, test_index in fold.split(X, y):
        
    
        print('fold n°: ', i)
        
        predictions_train = list()

        predictions_test = list()

        x_data, x_val = X.iloc[train_index], X.iloc[test_index]

        y_data, y_val = y.iloc[train_index], y.iloc[test_index]
        
        
        
        lgb_model_1 = LGBMClassifier(n_estimators = 1000, learning_rate = 0.09, scale_pos_weight = 7.3, boosting = 'dart', \
                            num_leaves = 60, importance_type = 'gain', reg_lambda = 0.5,
                          seed = 40)
        
        
        lgb_model_1.fit(x_data, y_data, categorical_feature = cat_cols, eval_set= [(x_val, y_val)],  verbose = 0)
        
    
        y_train_pred_lgb1 = lgb_model_1.predict_proba(x_data)
        
        y_val_pred_lgb1 = lgb_model_1.predict_proba(x_val)
        
        y_test_pred_lgb1 = lgb_model_1.predict_proba(test_new)
        
        
        
        
        lgb_model_2 = LGBMClassifier(n_estimators = 1000, learning_rate = 0.09, scale_pos_weight = 7.3, boosting = 'dart', \
                            num_leaves = 60, importance_type = 'gain', reg_alpha = 0.5, feature_fraction = 0.9,
                            seed = 40)
        
        lgb_model_2.fit(x_data, y_data, categorical_feature = cat_cols, eval_set= [(x_val, y_val)],  verbose = 0)
        
    
        y_train_pred_lgb2 = lgb_model_2.predict_proba(x_data)
        
        y_val_pred_lgb2 = lgb_model_2.predict_proba(x_val)
        
        y_test_pred_lgb2 = lgb_model_2.predict_proba(test_new)
    
    
    
        y_train_pred = ((1.5*(y_train_pred_lgb1)) + (1.0*(y_train_pred_lgb2)))/2
        
        y_val_pred = ((1.5*(y_val_pred_lgb1)) + (1.0*(y_val_pred_lgb2)))/2
        
        y_test_pred = ((1.5*(y_test_pred_lgb1)) + (1.0*(y_test_pred_lgb2)))/2
        
        
        y_train_pred = np.argmax(y_train_pred, axis = 1).tolist()
        
        y_val_pred = np.argmax(y_val_pred, axis = 1).tolist()
        
        y_test_pred = np.argmax(y_test_pred, axis = 1).tolist()
        
        
        
        
        
        predictions.append(y_test_pred)
        
        
        
    
        
        print('f1_binary train', f1_score(y_data, y_train_pred, average = 'binary'))
        print('f1_binary test',  f1_score(y_val, y_val_pred, average = 'binary'))
        
        print('f1_macro train', f1_score(y_data, y_train_pred, average = 'macro'))
        print('f1_macro test',  f1_score(y_val, y_val_pred, average = 'macro'))
    
        print('accuracy train', accuracy_score(y_data, y_train_pred))
        print('accuracy test',  accuracy_score(y_val, y_val_pred))
    
        print('balanced accuracy train', balanced_accuracy_score(y_data, y_train_pred))
        print('balanced accuracy test',  balanced_accuracy_score(y_val, y_val_pred))
    
        print('mcc train', matthews_corrcoef(y_data, y_train_pred))
        print('mcc test', matthews_corrcoef(y_val, y_val_pred))
    
    
    
        f1_bin.append(f1_score(y_val, y_val_pred, average = 'binary'))
        
        f1_mac.append(f1_score(y_val, y_val_pred, average = 'macro'))
        
        acc.append(accuracy_score(y_val, y_val_pred))
    
        bal_acc.append(balanced_accuracy_score(y_val, y_val_pred))
        
        mcc.append(matthews_corrcoef(y_val, y_val_pred))


        i = i + 1
        
    print('Mean f1 binary test: ', np.mean(f1_bin)) 
    
    print('Mean f1 macro test: ', np.mean(f1_mac))

    print('Mean Acc test: ', np.mean(acc))

    print('Mean Bal Acc test: ', np.mean(bal_acc))
    
    print('Mean Mcc test: ', np.mean(mcc))
    
    
    return predictions

In [31]:
predictions = ensemb_crossval()

fold n°:  1
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
f1_binary train 0.793939393939394
f1_binary test 0.40707964601769914
f1_macro train 0.8785335186810082
f1_macro test 0.655152726234656
accuracy train 0.9374482568300985
accuracy test 0.8336092715231788
balanced accuracy train 0.9644388662273821
balanced accuracy test 0.6791351737113569
mcc train 0.7819669277779997
mcc test 0.3171331605759981
fold n°:  2
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
f1_binary train 0.7990240927111923
f1_binary test 0.43030303030303035
f1_macro train 0.8816655425371485
f1_macro test 0.6700891949214097
accuracy train 0.9393800018397571
accuracy test 0.8443708609271523
balanced accuracy train 0.9655370777115364
balanced accuracy test 0.6912057611833782
mcc train 0.7870547161786969
mcc test 0.3451103004216974
fold n°:  3
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


...

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
f1_binary train 0.7969558599695586
f1_binary test 0.3626062322946175
f1_macro train 0.8804089046827259
f1_macro test 0.6267708815375173
accuracy train 0.9386440989789347
accuracy test 0.8137417218543046
balanced accuracy train 0.9651223593390503
balanced accuracy test 0.651852281815133
mcc train 0.7850095852678267
mcc test 0.26271676801486593
fold n°:  9
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
f1_binary train 0.7945371775417298
f1_binary test 0.3988439306358381
f1_macro train 0.8789187215837884
f1_macro test 0.6491804194241994
accuracy train 0.9377242204029068
accuracy test 0.8278145695364238
balanced accuracy train 0.9645994561807154
balanced accuracy test 0.6746252870004902
mcc train 0.7825904566891053
mcc test 0.30628840124725043
fold n°:  10
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\users\sourabh.jain\appdata\local\programs\python\python37\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Gender', 'Gender_age_risk', 'LGA_Name', 'Policy_brackets', 'ProductName', 'State', 'State_Statezone', 'State_zone', 'Subject_Car_Make', 'age_group', 'color_group', 'risk_index']
  _log_warning('categorical_feature in Dataset 

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
f1_binary train 0.791779993955878
f1_binary test 0.35014836795252224
f1_macro train 0.8772027173468027
f1_macro test 0.6223539143566175
accuracy train 0.936626195732156
accuracy test 0.8185584092792046
balanced accuracy train 0.9639719723907132
balanced accuracy test 0.6408305734138581
mcc train 0.7798123732671756
mcc test 0.2503741957489923
Mean f1 binary test:  0.3825517993052173
Mean f1 macro test:  0.6404046667238577
Mean Acc test:  0.8253161058285826
Mean Bal Acc test:  0.6628435222584402
Mean Mcc test:  0.28774675410352


In [33]:
feature_importances = pd.DataFrame({'feature': list(X.columns), 'importance': importances}).sort_values('importance', 
                                    ascending = True)

In [34]:
feature_importances

,feature,importance
25,Brown,70.045472
26,Green,95.090442
22,Ash,127.165542
31,Policy_brackets,130.816570
23,Red,155.423017
38,State_Statezone,170.997456
28,White,336.754270
20,Gold,455.101726
27,Wine,564.777865
32,State_zone,658.140832


In [35]:
test_predictions = np.array(predictions)

import scipy
test_predictions = list(scipy.stats.mode(test_predictions)[0][0])

In [36]:
ss['target'] = test_predictions

In [39]:
ss.to_csv('submission18.csv',index=False)  

In [40]:
ss['target'].value_counts()

0    1064
1     138
Name: target, dtype: int64